In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score



In [35]:
#We shall begin by importing the 2 US bureau dataframes from each year that we are concerned with:

df1 = pd.read_excel("/Users/jacob/Documents/4YP data/US_BureaOfLaborStatistics/annualstats/national_M2019_dl.xlsx")
df2 = pd.read_excel("/Users/jacob/Documents/4YP data/US_BureaOfLaborStatistics/annualstats/national_M2014_dl.xlsx")


In [36]:
#We shall now drop the unimportant columns:

df1 = df1[["occ_code","tot_emp"]]
df1.columns= df1.columns.str.upper()
#df1 = df1.rename(columns={"occ_code": "OCC_CODE", "occ_title": "OCC_TITLE", "tot_emp": "TOT_EMP"}) #ensure consistency of labels
df2 = df2[["OCC_CODE","TOT_EMP"]]


#Now add a holder column for the data to be added to df1:

df1["TOT_EMP2"]=np.nan
df1.head()

,OCC_CODE,TOT_EMP,TOT_EMP2
0,00-0000,146875480,NaN
1,11-0000,8054120,NaN
2,11-1000,2658440,NaN
3,11-1010,205890,NaN
4,11-1011,205890,NaN


In [ ]:
''' IGNORE ME '''

#Now we drop duplicate rows on both dataframes:

df1 = df1.drop_duplicates(subset = ["OCC_CODE"],keep="first")
df1 = df1.drop_duplicates(subset = ["TOT_EMP"],keep="first")
df2 = df2.drop_duplicates(subset = ["OCC_CODE"],keep="first")
df2 = df2.drop_duplicates(subset = ["TOT_EMP"],keep="first")

df1.reset_index(drop=True,inplace=True)
df2.reset_index(drop=True,inplace=True)

In [37]:
df1 = df1.groupby('OCC_CODE').median().reset_index()
df2 = df2.groupby('OCC_CODE').median().reset_index()

In [ ]:
#We shall now focus on concatenating these dataframes

def concat_data(a,b,key):
    my_list = list(b["OCC_CODE"])
    for i in range(0,len(my_list)):
        if my_list[i] in list(a["OCC_CODE"]) == True:
            a.loc[a["OCC_CODE"] == my_list[i],"TOT_EMP2"] = list(b.loc[b["OCC_CODE"] == my_list[i],key])
        else:
            pass
    return a

concat_data(df1,df2,"TOT_EMP")

In [38]:
my_list = list(df1["OCC_CODE"])

for i in range(0,len(my_list)):
    if my_list[i] in list(df2["OCC_CODE"]):
        df1.loc[df1["OCC_CODE"] == my_list[i],"TOT_EMP2"] = df2.loc[df2["OCC_CODE"] == my_list[i],"TOT_EMP"].values[0]

df1.head()

,OCC_CODE,TOT_EMP,TOT_EMP2
0,00-0000,146875480,135128260.0
1,11-0000,8054120,6741640.0
2,11-1000,2658440,2351130.0
3,11-1010,205890,246240.0
4,11-1011,205890,246240.0


In [39]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1316 entries, 0 to 1315
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   OCC_CODE  1316 non-null   object 
 1   TOT_EMP   1316 non-null   int64  
 2   TOT_EMP2  1224 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 31.0+ KB


In [40]:
df1.dropna(inplace=True)
df1["CHANGE_EMP"] = df1["TOT_EMP"] - df1["TOT_EMP2"]
df1.head()

,OCC_CODE,TOT_EMP,TOT_EMP2,CHANGE_EMP
0,00-0000,146875480,135128260.0,11747220.0
1,11-0000,8054120,6741640.0,1312480.0
2,11-1000,2658440,2351130.0,307310.0
3,11-1010,205890,246240.0,-40350.0
4,11-1011,205890,246240.0,-40350.0


In [41]:
#Now lets work on adding the skill values:

df3 = pd.read_csv("/Users/jacob/Documents/4YP data/ONET_18-10-21/test.csv")

In [42]:
df3 = df3.drop(columns=['Title', 'Auto label value'])
df3.head()

,O*NET-SOC Code,Reading Comprehension,Active Listening,Writing,Speaking,Mathematics,Science,Critical Thinking,Active Learning,Learning Strategies,...,Troubleshooting,Repairing,Quality Control Analysis,Judgment and Decision Making,Systems Analysis,Systems Evaluation,Time Management,Management of Financial Resources,Management of Material Resources,Management of Personnel Resources
0,11-1011.00,4.75,4.88,4.38,4.88,3.62,1.12,4.75,4.75,3.88,...,0.00,0.0,1.00,5.75,5.38,5.12,4.75,5.50,4.75,5.38
1,11-1011.03,4.25,4.00,4.25,4.12,3.12,1.88,4.12,3.88,3.75,...,0.00,0.0,1.50,4.00,4.00,4.00,3.88,3.12,2.62,4.00
2,11-1021.00,4.00,4.00,3.88,4.00,2.50,1.12,4.00,3.62,3.25,...,1.38,0.0,2.12,3.75,3.00,3.12,3.75,3.38,3.25,3.88
3,11-2011.00,4.00,4.12,3.88,4.12,3.25,0.62,4.12,4.12,3.25,...,0.00,0.0,1.00,4.00,3.12,3.75,3.88,3.62,2.62,3.88
4,11-2021.00,4.25,4.12,3.88,4.12,3.12,1.50,4.25,4.12,3.50,...,0.00,0.0,1.38,4.00,3.75,3.75,3.75,3.75,2.75,3.88


In [43]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873 entries, 0 to 872
Data columns (total 36 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   O*NET-SOC Code                     873 non-null    object 
 1   Reading Comprehension              873 non-null    float64
 2   Active Listening                   873 non-null    float64
 3   Writing                            873 non-null    float64
 4   Speaking                           873 non-null    float64
 5   Mathematics                        873 non-null    float64
 6   Science                            873 non-null    float64
 7   Critical Thinking                  873 non-null    float64
 8   Active Learning                    873 non-null    float64
 9   Learning Strategies                873 non-null    float64
 10  Monitoring                         873 non-null    float64
 11  Social Perceptiveness              873 non-null    float64

In [44]:
def title_set(my_string):
    my_output = my_string[:-3]
    return my_output

df3.iloc[:,0] = df3.iloc[:,0].apply(title_set)

df3 = df3.groupby('O*NET-SOC Code').median().reset_index()

df3.head()
#title_set("11-1011.00")

,O*NET-SOC Code,Reading Comprehension,Active Listening,Writing,Speaking,Mathematics,Science,Critical Thinking,Active Learning,Learning Strategies,...,Troubleshooting,Repairing,Quality Control Analysis,Judgment and Decision Making,Systems Analysis,Systems Evaluation,Time Management,Management of Financial Resources,Management of Material Resources,Management of Personnel Resources
0,11-1011,4.50,4.44,4.315,4.50,3.37,1.50,4.435,4.315,3.815,...,0.00,0.0,1.25,4.875,4.69,4.56,4.315,4.31,3.685,4.69
1,11-1021,4.00,4.00,3.880,4.00,2.50,1.12,4.000,3.620,3.250,...,1.38,0.0,2.12,3.750,3.00,3.12,3.750,3.38,3.250,3.88
2,11-2011,4.00,4.12,3.880,4.12,3.25,0.62,4.120,4.120,3.250,...,0.00,0.0,1.00,4.000,3.12,3.75,3.880,3.62,2.620,3.88
3,11-2021,4.25,4.12,3.880,4.12,3.12,1.50,4.250,4.120,3.500,...,0.00,0.0,1.38,4.000,3.75,3.75,3.750,3.75,2.750,3.88
4,11-2022,4.00,4.00,4.000,4.12,3.25,0.62,4.000,3.880,3.750,...,0.00,0.0,1.38,4.000,3.75,3.88,3.880,3.62,3.000,4.00


In [45]:
#We now have got to deal with duplicate SOC codes

df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759 entries, 0 to 758
Data columns (total 36 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   O*NET-SOC Code                     759 non-null    object 
 1   Reading Comprehension              759 non-null    float64
 2   Active Listening                   759 non-null    float64
 3   Writing                            759 non-null    float64
 4   Speaking                           759 non-null    float64
 5   Mathematics                        759 non-null    float64
 6   Science                            759 non-null    float64
 7   Critical Thinking                  759 non-null    float64
 8   Active Learning                    759 non-null    float64
 9   Learning Strategies                759 non-null    float64
 10  Monitoring                         759 non-null    float64
 11  Social Perceptiveness              759 non-null    float64

In [46]:
# We are all setup to concatenate employment change to each of the skill variables!

df3["Change in Employment"] = np.nan

my_list = list(df3["O*NET-SOC Code"])

for i in range(0,len(my_list)):
    if my_list[i] in list(df1["OCC_CODE"]):
        df3.loc[df3["O*NET-SOC Code"] == my_list[i],"Change in Employment"] = df1.loc[df1["OCC_CODE"] == my_list[i],"CHANGE_EMP"].values[0]

df3 = df3.dropna(subset=['Change in Employment'])
df3.head()

,O*NET-SOC Code,Reading Comprehension,Active Listening,Writing,Speaking,Mathematics,Science,Critical Thinking,Active Learning,Learning Strategies,...,Repairing,Quality Control Analysis,Judgment and Decision Making,Systems Analysis,Systems Evaluation,Time Management,Management of Financial Resources,Management of Material Resources,Management of Personnel Resources,Change in Employment
0,11-1011,4.50,4.44,4.315,4.50,3.37,1.50,4.435,4.315,3.815,...,0.0,1.25,4.875,4.69,4.56,4.315,4.31,3.685,4.69,-40350.0
1,11-1021,4.00,4.00,3.880,4.00,2.50,1.12,4.000,3.620,3.250,...,0.0,2.12,3.750,3.00,3.12,3.750,3.38,3.250,3.88,350410.0
2,11-2011,4.00,4.12,3.880,4.12,3.25,0.62,4.120,4.120,3.250,...,0.0,1.00,4.000,3.12,3.75,3.880,3.62,2.620,3.88,-4240.0
3,11-2021,4.25,4.12,3.880,4.12,3.12,1.50,4.250,4.120,3.500,...,0.0,1.38,4.000,3.75,3.75,3.750,3.75,2.750,3.88,79190.0
4,11-2022,4.00,4.00,4.000,4.12,3.25,0.62,4.000,3.880,3.750,...,0.0,1.38,4.000,3.75,3.88,3.880,3.62,3.000,4.00,43680.0


In [48]:
df3.to_csv("regressordata.csv",index=False)

In [ ]:
list(df3["O*NET-SOC Code"])

In [ ]:
list(df1["OCC_CODE"])

In [ ]:
df3.loc[df3["O*NET-SOC Code"] == my_list[1],"Change in Employment"] = df1.loc[df1["OCC_CODE"] == my_list[1],"CHANGE_EMP"].values[0]

In [47]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 661 entries, 0 to 758
Data columns (total 37 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   O*NET-SOC Code                     661 non-null    object 
 1   Reading Comprehension              661 non-null    float64
 2   Active Listening                   661 non-null    float64
 3   Writing                            661 non-null    float64
 4   Speaking                           661 non-null    float64
 5   Mathematics                        661 non-null    float64
 6   Science                            661 non-null    float64
 7   Critical Thinking                  661 non-null    float64
 8   Active Learning                    661 non-null    float64
 9   Learning Strategies                661 non-null    float64
 10  Monitoring                         661 non-null    float64
 11  Social Perceptiveness              661 non-null    float64

In [26]:
list(df3["O*NET-SOC Code"])


['11-1011',
 '11-1021',
 '11-2011',
 '11-2021',
 '11-2022',
 '11-3012',
 '11-3021',
 '11-3031',
 '11-3051',
 '11-3061',
 '11-3071',
 '11-3111',
 '11-3121',
 '11-3131',
 '11-9013',
 '11-9021',
 '11-9031',
 '11-9032',
 '11-9033',
 '11-9041',
 '11-9051',
 '11-9071',
 '11-9081',
 '11-9111',
 '11-9121',
 '11-9131',
 '11-9141',
 '11-9151',
 '11-9161',
 '11-9171',
 '11-9179',
 '11-9199',
 '13-1011',
 '13-1021',
 '13-1022',
 '13-1023',
 '13-1031',
 '13-1032',
 '13-1041',
 '13-1051',
 '13-1071',
 '13-1074',
 '13-1075',
 '13-1081',
 '13-1111',
 '13-1121',
 '13-1131',
 '13-1141',
 '13-1151',
 '13-1161',
 '13-1199',
 '13-2011',
 '13-2023',
 '13-2031',
 '13-2041',
 '13-2052',
 '13-2053',
 '13-2061',
 '13-2071',
 '13-2072',
 '13-2081',
 '13-2082',
 '13-2099',
 '15-1211',
 '15-1212',
 '15-1221',
 '15-1231',
 '15-1232',
 '15-1241',
 '15-1242',
 '15-1243',
 '15-1244',
 '15-1251',
 '15-1253',
 '15-1254',
 '15-1255',
 '15-1299',
 '15-2011',
 '15-2021',
 '15-2031',
 '15-2041',
 '15-2051',
 '15-2099',
 '17

In [29]:
list(df1["OCC_CODE"])

['00-0000',
 '11-0000',
 '11-1000',
 '11-1010',
 '11-1011',
 '11-1020',
 '11-1021',
 '11-1030',
 '11-1031',
 '11-2000',
 '11-2010',
 '11-2011',
 '11-2020',
 '11-2021',
 '11-2022',
 '11-2030']